## Introduction

- MNIST 是一個手寫數字的圖像資料集，我們將用於圖像辨識的範例之中。

- MNIST 資料集常用於機器學習訓練和測試的教學。
  資料集包含 60000 個訓練圖片和 10000 個測試圖片，每個圖片大小是 28 * 28 像素。

  <img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png">
  

## 0. 引用相關模組

In [ ]:
'''Trains a simple convnet on the MNIST dataset.'''


from __future__ import print_function
import tensorflow.keras as keras

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import requests
# import pandas as pd
from numpy import genfromtxt
import zipfile
import numpy as np
import matplotlib.pyplot as plt


# input image dimensions
img_rows, img_cols = 28, 28
batch_size = 128
num_classes = 10
epochs = 12


## 1. 資料讀取(Data reading)

- 自Scidm讀取 MNIST 資料集，將資料解壓縮，並分為訓練資料集及測試資料集
- 宣告函式：讀取 MNIST 並格式化為 28*28 陣列

In [ ]:
# load mnist method 1
def download_unzip(url):
    print("[1-1] Download MNIST from scidm.nchc.org.tw")
    local_filename = url.split('/')[-1]
    r = requests.get(url)

    # NOTE the stream=True parameter below
    #with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("[Finish] Downloaded")
    print("[1-2] unzip file")
    with zipfile.ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall("./")
    print("[Finish] generating mnist_test.csv & mnist_train.csv")
    return local_filename

mnist_in_csv_zip = "https://scidm.nchc.org.tw/dataset/ef890176-6fd9-499d-9687-5fe2863c6941/resource/ce2b188c-bb3e-41ab-8dfc-d9b6e665fbd4/nchcproxy/mnist-in-csv.zip"
download_unzip(mnist_in_csv_zip)

print("[1-3] loading mnist file into data structure ")

mnist_test = genfromtxt("mnist_test.csv", delimiter=',')    
mnist_train = genfromtxt("mnist_train.csv", delimiter=',')    

x_train= mnist_train[1:,1:]
y_train= mnist_train[1:,0:1]
x_test = mnist_test[1:,1:]
y_test = mnist_test[1:,0:1]



In [ ]:
for i in range(10):
    print("label: "+str(y_train[i]))
    show_img = x_train[i].reshape(28,28)
    plt.imshow(show_img,aspect="auto")
    plt.show()

# 2.資料處理(Data preprocessing)：

## 2-1 image reshape
- 圖像資料：
  將資料下載後儲存於在本地端資料夾，本地文件夾中保存圖片：圖像儲存為 28 * 28 = 784 長度的向量。
- CNN：
  image reshape (60000, 28, 28, 1): CNN 因為必須先進行卷積與池化 (Max-Pool) 運算, 所以必須保留影像的維度. 因此 60,000 筆轉換成一筆成 28 (長) x 28(寬) x 1(高) 的影像單位.

## 2-2 one-hot encoding
- 標籤資料：
  使用有效編碼(One-Hot Encoding)編碼格式儲存，用於表示名目尺度的一種編碼格式，使用時較有效率。

In [ ]:
print("[2-1 ] reshape data into to (n, 28, 28, 1) ")

x_train = x_train.reshape(len(x_train), 28, 28,1)
x_test = x_test.reshape(len(x_test), 28, 28,1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print("x_train =shape=> {}".format(x_train.shape))
print("y_train =shape=> {}".format(y_train.shape))
print("x_test =shape=> {}".format(x_test.shape))
print("y_test =shape=> {}".format(y_test.shape))

print("[Finish] converted ")

print("[2-2] One-Hot Encoding ")
# convert class vectors to binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("[Finish] converted ")

In [ ]:
for i in range(1000,1006):
    print("label: "+str(y_train[i]))
    show_img = x_train[i].reshape(28,28)
    plt.imshow(show_img,aspect="auto")
    plt.show()

# 3.  CNN運算 (Convolution) 

[卷積神經網路(Convolutional Neural Network, CNN)](https://zh.wikipedia.org/wiki/%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C)

## 3-1. 建立卷積層與池化層 
[卷積層&池化層](https://zh.wikipedia.org/wiki/%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C#%E5%8D%B7%E7%A9%8D%E5%B1%A4)

## 3-2. 建立神經網路 
- 建立平坦層 
  - 下面程式碼建立平坦層, 將之前步驟已經建立的池化層2, 共有 36 個 7x7 維度的影像轉換成 1 維向量, 長度是 36x7x7 = 1764, 也就是對應到 1764 個神經元: 
- 建立 Hidden layer 
- 建立輸出層 
  - 最後建立輸出層, 共有 10 個神經元, 對應到 0~9 共 10 個數字. 並使用 softmax 激活函數 進行轉換 (softmax 函數可以將神經元的輸出轉換成每一個數字的機率): 

## 3-3. 查看模型的摘要 

In [ ]:
print("[3-1] initial CN ")

model = Sequential()
# Create CN layer 1  
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#? Create Max-Pool 1
#? model.add(MaxPooling2D(pool_size=(2,2)))  

# Create CN layer 2  
model.add(Conv2D(64, (3, 3), activation='relu'))
# Create Max-Pool 2 
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add Dropout layer  
model.add(Dropout(0.25))

print("[3-2] create NN layer ")
# Add Flatten layer
model.add(Flatten())
# Add Hidden layer
model.add(Dense(128, activation='relu'))
# Add Dropout layer  
model.add(Dropout(0.5))

# Add Output by softmax
model.add(Dense(num_classes, activation='softmax'))
print("[3-3] check summary ")
model.summary() 

# 4.  用CNN模組進行訓練
定義訓練並進行訓練 

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
print("[Finish] keras model success")
print("[4] evaluate...")
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("[done] ")